# CHB MIT DATABASE

1. Load edf data
2. define class sz/nsz
3. insert class cols
4. concat
5. windowing
6. ensemple learn resnet152, inceptionv3, incep+resnet
7. define OHEM loss
8. build network
9. evaluate
10. calculate for confussion matrices (f1-score etc.)

In [87]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


## GLOBAL VARIABLES

In [88]:
cwd = os.getcwd()
FREQ = 256
database_path = 'Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/'
filtered_database_path = 'Dataset/CHB-MIT/Filtered-chb-mit/'
filted_db_parquet_path = "Dataset/CHB-MIT/dataframe-parquet"
edf_file_type = ".edf"
patient_one_path = 'chb04/'
summary_txt_file_type = "-summary.txt"
cwd

'/Users/niklashjort/Desktop/Notes/Speciale/projects'

In [89]:
def get_all_patient_folder_names(database_path):
    folders = os.listdir(database_path)
    patient_folder_names = [(database_path + "/" + x) for x in folders if (x.find(".DS_Store") == -1)]
    return patient_folder_names

def get_all_file_names(directory):
    files = os.listdir(directory)
    edfFileNameList = [(directory + "/" + x) for x in files if (x.endswith(edf_file_type))]
    summary_info_file_name = [(directory + "/" + x) for x in files if (x.endswith(summary_txt_file_type))]
    return (summary_info_file_name[0], edfFileNameList)

In [90]:
# Testing:
summary_file_path, edf_file_paths = get_all_file_names(get_all_patient_folder_names(filtered_database_path)[0])

print(f"summary file name: {summary_file_path} edf file name: {edf_file_paths[0]}")

summary file name: Dataset/CHB-MIT/Filtered-chb-mit//chb20/chb20-summary.txt edf file name: Dataset/CHB-MIT/Filtered-chb-mit//chb20/chb20_59.edf


In [313]:
def ReadEdfFile(FileName, print_reader_info = False):
    try:
        if(print_reader_info):
            data = mne.io.read_raw_edf(FileName)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw
        else:
            data = mne.io.read_raw_edf(FileName, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw
    except Exception as e:
        pass

## PROBLEM! bad channel: solution find conda mne and modify package.

In [309]:
print(mne.__version__)
bla = "Dataset/CHB-MIT/Filtered-chb-mit//chb18/chb18_01.edf"
ReadEdfFile(bla)

0.16.2
Dataset/CHB-MIT/Filtered-chb-mit//chb18/chb18_01.edf


In [138]:
# Testing 
test_df = ReadEdfFile(edf_file_paths[8], print_reader_info=False)
test_df.head()

,FP1-F7,F7-T7,T7-P7,P7-O1,.-0,FP1-F3,F3-C3,C3-P3,P3-O1,.-1,...,FP2-F8,F8-T8,T8-P8,P8-O2,.-4,P7-T7,T7-FT9,FT9-FT10,FT10-T8,STI 014
0,5.919414e-05,6.388278e-05,-3.809524e-05,-4.825397e-05,1.953602e-07,8.400488e-06,5.489621e-05,-3.711844e-06,-2.285714e-05,1.953602e-07,...,1.738706e-05,4.004884e-05,-2.793651e-05,-5.919414e-05,1.953602e-07,3.848596e-05,-3.711844e-06,-1.113553e-05,-1.894994e-05,131045.0
1,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
2,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,-1.953602e-07,0.0
3,1.953602e-07,1.953602e-07,1.953602e-07,1.758242e-06,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.367521e-06,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,9.768010e-07,-9.768010e-07,0.0
4,1.953602e-07,5.860806e-07,1.953602e-07,-1.758242e-06,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,-1.367521e-06,1.953602e-07,...,1.953602e-07,5.860806e-07,1.953602e-07,-1.953602e-07,1.953602e-07,1.953602e-07,-9.768010e-07,-1.758242e-06,3.321123e-06,0.0


## Memory Usage & Loading Time

Edf files takes a lot of memory (look below)

In [93]:
# print(test_df.info(memory_usage='deep'))
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

print(mem_usage(test_df))

189.71 MB


## Add col ['timems', class] - Reduce dtypes - Compress + Save

Adding cols

In [94]:
pd.io.parquet.get_engine('auto').__class__

pandas.io.parquet.FastParquetImpl

Reduce dtypes, save memory, uptimize loading times

In [95]:
def downcast_dtypes(df):
    _start = df.memory_usage(deep=True).sum() / 1024 ** 2
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    _end = df.memory_usage(deep=True).sum() / 1024 ** 2
    saved_time = (_start - _end) / _start * 100
    #print(f"Saved: {saved_time:.2f}%")
    return df

In [96]:
# Testing:
test_df = downcast_dtypes(test_df)

Insert time ms in dataframe function

In [170]:
def read_format_info_file(txt_summary_file_path):
    str_container = ""
    with open(txt_summary_file_path, 'r') as f:
        for line in f:
            str_container += str(line).replace("\n", "<br>")

    formatted_str = re.findall('(.*?)<br><br>', str_container)
    bla = [x.group() for x in re.finditer('(.*?)<br><br>', str_container)]
    for index, line in enumerate(formatted_str):
        if re.match('(^Channels in EDF Files:|^Channels changed:)', line):
            formatted_str.remove(formatted_str[index])
        else:
            pass
    return formatted_str


In [171]:
# Testing:
read_format_info_file(filtered_database_path + "/chb20/chb20-summary.txt")

['Data Sampling Rate: 256 Hz<br>*************************',
 'File Name: chb20_01.edf<br>File Start Time: 16:55:08<br>File End Time: 17:55:14<br>Number of Seizures in File: 0',
 'File Name: chb20_02.edf<br>File Start Time: 17:55:44<br>File End Time: 18:55:44<br>Number of Seizures in File: 0',
 'File Name: chb20_03.edf<br>File Start Time: 18:55:51<br>File End Time: 19:55:51<br>Number of Seizures in File: 0',
 'File Name: chb20_04.edf<br>File Start Time: 19:55:59<br>File End Time: 20:55:59<br>Number of Seizures in File: 0',
 'File Name: chb20_05.edf<br>File Start Time: 20:56:06<br>File End Time: 21:56:06<br>Number of Seizures in File: 0',
 'File Name: chb20_06.edf<br>File Start Time: 21:56:13<br>File End Time: 22:56:13<br>Number of Seizures in File: 0',
 'File Name: chb20_07.edf<br>File Start Time: 22:56:19<br>File End Time: 23:56:19<br>Number of Seizures in File: 0',
 'File Name: chb20_08.edf<br>File Start Time: 23:56:26<br>File End Time: 24:56:26<br>Number of Seizures in File: 0',
 'Fi

In [98]:
# Testing:
for index, obj in enumerate(read_format_info_file(summary_file_path)):
    print(f"index: {index} line: {obj}")

index: 0 line: File Name: chb20_02.edf<br>File Start Time: 17:55:44<br>File End Time: 18:55:44<br>Number of Seizures in File: 0
index: 1 line: File Name: chb20_04.edf<br>File Start Time: 19:55:59<br>File End Time: 20:55:59<br>Number of Seizures in File: 0
index: 2 line: File Name: chb20_06.edf<br>File Start Time: 21:56:13<br>File End Time: 22:56:13<br>Number of Seizures in File: 0
index: 3 line: File Name: chb20_08.edf<br>File Start Time: 23:56:26<br>File End Time: 24:56:26<br>Number of Seizures in File: 0
index: 4 line: File Name: chb20_12.edf<br>File Start Time: 03:56:53<br>File End Time: 4:56:53<br>Number of Seizures in File: 1<br>Seizure 1 Start Time: 94 seconds<br>Seizure 1 End Time: 123 seconds
index: 5 line: File Name: chb20_14.edf<br>File Start Time: 05:57:07<br>File End Time: 6:57:07<br>Number of Seizures in File: 1<br>Seizure 1 Start Time: 1971 seconds<br>Seizure 1 End Time: 2009 seconds
index: 6 line: File Name: chb20_16.edf<br>File Start Time: 07:57:20<br>File End Time: 8:5

Get sz information for [patient]-summary.txt files. </br>
Then get information on seizures timestamp and the file name </br>

In [319]:
class FileInformationContainer:
    def __init__(self, information_str):
        self.information_str = self.clean_string(information_str)
        self.file_name = self.set_filename()
        self.time_start = self.set_file_time_start_ms()
        self.sz_info = self.set_sz_information()
        
    def clean_string(self, uncleaned_str):
        return uncleaned_str.replace("<br>", " ")

    def set_filename(self):
        filename_found = re.match(r"^File Name: (.+?).edf", self.information_str)
        if filename_found:
            return filename_found.group(1)
        else:
            print(f"{self.file_name} failed get_filename")
            return "filename not found"
    
    def get_milli_sec(self, time_str):
        """Get Seconds from time."""
        h, m, s = time_str.split(':')
        return (int(h) * 3600 + int(m) * 60 + int(s)) * 1000

    def set_file_time_start_ms(self):
        time_start_found = re.match(r".*File Start Time: (.*?) File", self.information_str)
        if time_start_found:
            try:
                return self.get_milli_sec(time_start_found.group(1))
            except Exception as e :
                print(f"{self.file_name}: error {e} cannot convert to ms time")
                return f"{e}"
        else:
            print(f"{self.file_name} failed get_file_time_start_ms")
            return "time start not found"
    
    def get_sz_count(self):
        sz_count = 0
        count_found = re.search(r".*Seizures in File: (.*?) Seizure", self.information_str)
        if count_found:
            sz_count = count_found.group(1)
        if int(sz_count) > 0:
            return int(sz_count)
        else:
            return 0
        

    def set_sz_information(self):
        if(type(self.get_sz_count()) != None and self.get_sz_count() > 0):
            try:
                pattern = re.compile(r"Seizure [1-9] (?P<state>[?:Start|End]+) Time: (?P<Sec>[0-9]+)")
                myList = [m.groupdict() for m in pattern.finditer(self.information_str)]
                for item in myList:
                    converted_time = int(item.get("Sec")) * 1000
                    item["Sec"] = converted_time
                formatted = []
                for i in range(0, len(myList), 2):
                    formatted.append({"sz_start" : myList[i]["Sec"], "sz_end" : myList[i+1]["Sec"]})
                return formatted
            except Exception as e:
                print(f"set_sz_information failed at file: {self.file_name} with the following exception: {e}")
        else:
            return []

    def get(self):
        return self.information_str

In [100]:
# Testing:
w_no_sz_str = FileInformationContainer(read_format_info_file(summary_file_path)[1])
print(f"Object with sz \n info-str: {w_no_sz_str.information_str}\n file-name: {w_no_sz_str.file_name}\n time-start: {w_no_sz_str.time_start}\n sz-information: {w_no_sz_str.sz_info} \n")
print("-------------------------------------")
w_sz_str = FileInformationContainer(read_format_info_file(summary_file_path)[4])
print(f"Object with sz \n info-str: {w_sz_str.information_str}\n file-name: {w_sz_str.file_name}\n time-start: {w_sz_str.time_start}\n sz-information: {w_sz_str.sz_info}")

Object with sz 
 info-str: File Name: chb20_04.edf File Start Time: 19:55:59 File End Time: 20:55:59 Number of Seizures in File: 0
 file-name: chb20_04
 time-start: 71759000
 sz-information: [] 

-------------------------------------
Object with sz 
 info-str: File Name: chb20_12.edf File Start Time: 03:56:53 File End Time: 4:56:53 Number of Seizures in File: 1 Seizure 1 Start Time: 94 seconds Seizure 1 End Time: 123 seconds
 file-name: chb20_12
 time-start: 14213000
 sz-information: [{'sz_start': 94000, 'sz_end': 123000}]


In [110]:
# Creating Test object:
obj = FileInformationContainer(read_format_info_file(summary_file_path)[4])
print(obj.file_name)
obj.time_start

chb20_12


14213000

Insert timestamp in files based of freq

In [102]:
def insert_time_stamp(dataframe, file_start_time):
    period_row_increment_value =  (1 / 256) * 1000
    dataframe.insert(0, "timestamp", [file_start_time + i * period_row_increment_value for i in dataframe.index])
    
    
insert_time_stamp(test_df, obj.time_start)

In [103]:
# Testing:
test_df.head()

,timestamp,FP1-F7,F7-T7,T7-P7,P7-O1,.-0,FP1-F3,F3-C3,C3-P3,P3-O1,...,FP2-F8,F8-T8,T8-P8,P8-O2,.-4,P7-T7,T7-FT9,FT9-FT10,FT10-T8,STI 014
0,2.864000e+07,1.387057e-05,-3.692308e-05,-4.493284e-06,-2.930403e-06,1.953602e-07,2.324786e-05,-5.919414e-05,6.661782e-05,-6.114774e-05,...,-2.402930e-05,1.953602e-07,5.333333e-05,2.520147e-05,1.953602e-07,4.884005e-06,-7.619048e-06,-4.395604e-05,6.075702e-05,53.0
1,2.864000e+07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
2,2.864001e+07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,-1.953602e-07,0.0
3,2.864001e+07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
4,2.864002e+07,1.953602e-07,1.953602e-07,-1.953602e-07,-1.758242e-06,1.953602e-07,1.953602e-07,-5.860806e-07,1.953602e-07,-1.758242e-06,...,-1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,-3.321123e-06,-1.953602e-07,4.102564e-06,0.0


insert class column based on the below frow paper: </br>
Interictal = normal state </br>
Preictal I = 30 min. before sz </br>
Preictal II = 10 min. before sz </br>
Seizure = sz_start -> sz_end 

In [238]:
def insert_class_col(dataframe, sz_info_list):
    
    if "class" not in dataframe.columns:
        dataframe.insert(0, "class", np.nan)

    if len(sz_info_list) == 0:
        dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal I") & (dataframe['class'] != "Preictal II"), "class"] = "Interictal"
    else:
        for item in sz_info_list:
            sz_start = item["sz_start"]
            sz_end = item["sz_end"]
            #print(f"hihi: {dataframe.iloc[0]['timestamp']}")
            actual_sz_start = dataframe.iloc[0]['timestamp'] + sz_start
            preictal_one_start = actual_sz_start - (30 * 60 * 1000)
            preictal_two_start = actual_sz_start - (10 * 60 * 1000)
            actual_sz_end = dataframe.iloc[0]['timestamp'] + sz_end
            #print(f"sz: {sz_start} end: {sz_end} prei1: {preictal_one_start} prei2: {preictal_two_start} actsz: {actual_sz_start} actend: {actual_sz_end}")
            dataframe['timestamp'] = pd.to_numeric(dataframe['timestamp'])

            #INSERTING INTERICTAL
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal I") & (dataframe['class'] != "Preictal II") & (dataframe['timestamp'] > actual_sz_end) | (dataframe['timestamp'] < actual_sz_start), "class"] = "Interictal"

            #INSERTING PREICTAL I
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal II")  & (dataframe['timestamp'] >= preictal_one_start) & (dataframe['timestamp'] < preictal_two_start), "class"] = "Preictal I"

            #INSERTING PREICTAL II
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['timestamp'] >= preictal_two_start) & (dataframe['timestamp'] < actual_sz_start), "class"] = "Preictal II"

            #INSERTING SEIZURE CLASS
            dataframe.loc[(dataframe['timestamp'] >= actual_sz_start) & (dataframe['timestamp'] < actual_sz_end), "class"] = "seizure"

            
           # print(f"na: {dataframe['class'].isna().sum()}")
            #print(f"datapoint: {((actual_sz_end - actual_sz_start)/3906) * 1000}")
            #print(dataframe["class"].value_counts())
    

    

In [105]:
#test_df = test_df.drop(columns="class")

In [106]:
# Testing:
filename = obj.file_name
count = obj.get_sz_count()
start = obj.sz_info[0]["sz_start"]
end = obj.sz_info[0]["sz_end"]
print(f"filename: {filename} sz_count: {count} sz_start: {start} sz_end: {end}")
insert_class_col(test_df, obj.sz_info)
test_df['class'].value_counts()

filename: chb20_16 sz_count: 1 sz_start: 2226000 sz_end: 2261000
[{'sz_start': 2226000, 'sz_end': 2261000}]
inside
inside3
sz: 2226000 end: 2261000 prei1: 29066000.0 prei2: 30266000.0 actsz: 30866000.0 actend: 33127000.0


seizure        318208
Preictal I     307200
Preictal II    153600
Interictal     109056
Name: class, dtype: int64

Missing "Preictal I" because seizure is very early from file time start...

Compress & save

In [142]:
def df_save_compress(path, filename, df):
    df.to_parquet(f"{path}{filename}.parquet.gzip", compression="gzip")
# Exampled
# df_save_compress(database_path, patient_one_path, "filename", ch01_df)

1. Load all files in paper. </br> 2. For each directory (patient) extract file information to object.</br> 3. Downcast datatypes. </br>4. Insert class and timestamp cols.</br> 5.Compress and save.

## Modifying and saving all dataframes to parquet format

In [327]:
save_parquet_path = "Dataset/CHB-MIT/dataframe-parquet/"

for patient in get_all_patient_folder_names(filtered_database_path):
    current_patient = patient
    info_txt_path, edf_files = get_all_file_names(current_patient)
    # read & extract information
    info_txt = read_format_info_file(info_txt_path)
    for line in info_txt[2:]:
        edf_info_container = FileInformationContainer(line)
        selected_edf_path = [x for x in edf_files if (edf_info_container.file_name in x)][0]
        edf_df = ReadEdfFile(selected_edf_path)
        if edf_df is not None:
            edf_df = downcast_dtypes(edf_df)
            insert_time_stamp(edf_df, edf_info_container.time_start)
            insert_class_col(edf_df, edf_info_container.sz_info)
            print(f"filename: {edf_info_container.file_name} classes: {edf_df['class'].value_counts()}")
            df_save_compress(save_parquet_path, edf_info_container.file_name, edf_df)

filename: chb20_02 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_03 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_04 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_05 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_06 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_07 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_08 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_11 classes: Interictal    921600
Name: class, dtype: int64
filename: chb20_12 classes: Interictal     890111
Preictal II     24064
seizure          7424
Name: class, dtype: int64
filename: chb20_13 classes: Interictal     450816
Preictal I     307200
Preictal II    153600
seizure          9984
Name: class, dtype: int64
filename: chb20_14 classes: Interictal     451071
Preictal I     307200
Preictal II    153600
seizure          9728
Name: class, dtype: in

# Read & load compressed dataframes

In [325]:
compressed_file_type = ".parquet.gzip"
compressed_df = pd.read_parquet(save_parquet_path + "chb18_02" + compressed_file_type)

In [326]:
compressed_df.head()

,class,timestamp,FP1-F7,F7-T7,T7-P7,P7-O1,.-0,FP1-F3,F3-C3,C3-P3,...,FP2-F8,F8-T8,T8-P8,P8-O2,.-4,P7-T7,T7-FT9,FT9-FT10,FT10-T8,STI 014
0,Interictal,6.809700e+07,-1.869597e-04,-3.028083e-05,4.512821e-05,-4.083028e-05,1.953602e-07,-1.666422e-04,-2.012210e-05,-1.953602e-07,...,-2.256410e-04,-1.855922e-05,-1.426129e-05,1.465201e-05,1.953602e-07,-4.473748e-05,-3.770452e-05,3.887668e-05,1.035409e-05,131058.0
1,Interictal,6.809700e+07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
2,Interictal,6.809701e+07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
3,Interictal,6.809701e+07,9.768009e-07,5.860806e-07,1.953602e-07,-1.953602e-07,1.953602e-07,1.758242e-06,1.953602e-07,-1.953602e-07,...,5.860806e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,0.0
4,Interictal,6.809702e+07,-9.768009e-07,1.953602e-07,-1.953602e-07,5.860806e-07,1.953602e-07,-9.768009e-07,1.953602e-07,5.860806e-07,...,1.953602e-07,5.860806e-07,-1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,1.953602e-07,1.953602e-07,5.860806e-07,0.0
